## Lab 2: Zombie Apocalypse 🧟
__Version 1__

The city is under attack from zombies and several vaccines have been proposed but the government isn't sure which one to use. Your job is to create a simulation to test the vaccines and analyze them using inferential statistics so you can make a recommendation.

A sample of the population has been created in the file "lab2-sample.csv". The data contain the status of an individual (1 = infected, 0 = non-infected), and their x and y coordinates in the city. Scientists have determined that a person has a 30% chance per day of being infected by a zombie within a 5 unit radius (all distances have been converted to standard units).

Follow the prompts below that guide you through your research. When you are finished submit an HTML version of your notebook.

Good luck, the city is counting on you!

---

Import pandas, lab2lib, and any other modules you need here.

In [1]:
import pandas as pd
import matplotlib as plt
import scipy.stats as stats
from statsmodels.stats import proportion
import numpy as np

Load the `lab2-sample.csv` data into a dataframe and get familiar with it.

In [2]:
sample = pd.read_csv('lab2-sample.csv')

sample.describe()

sample.sort_values(by = 'status', ascending = False)

,status,x,y
34,1,12.125416,12.605890
142,1,9.753383,24.630488
2,0,15.560043,28.581505
0,0,46.598301,42.323014
4,0,18.555827,46.410041
...,...,...,...
195,0,25.119138,24.119612
196,0,45.306712,18.898339
197,0,31.170071,37.228473
198,0,29.325565,29.565473


---

The first thing you'll need is a Python function to simulate a single day. Create a function which takes the following parameters:
1. A dataframe of the population to simulate
2. The infection radius distance (default to 5)
3. The infection probability of non-vaccinated people (default to .3)
4. The infection probability of vaccinated people (default to -1 to indicate no vaccine)

The function should iterate through all non-infected people (status == 0). For each person it should find all infected people within the infection radius and generate a random number to determine if an infection event should take place. If the vaccinated infection probability is not -1 and the population dataframe includes a `vaccinated` column and the non-infected person is vaccinated (`vaccinated` == 1) the infection probablity of vaccinated people should be used to determin the infection event, otherwise the infection probability of non-vaccinated people should be used. If infected, a person's status should be set to 1. The function should return the population dataframe.

In [3]:
def day(pop, r = 5, p_i = 0.3, p_v = -1):
    groups = [x for x in pop.groupby('status')]
    for idx, p in groups[0][1].iterrows():
        
        pvac = p_v if 'vaccinated' in pop.columns and p_v != -1 and p['vaccinated'] == 1 else p_i 
        
        for _, z in groups[1][1].iterrows():
            
            if (p.x - z.x)**2 + (p.y - z.y)**2 < r ** 2: 
                if np.random.random() < pvac:
                    pop.at[idx, 'status'] = 1
                    break
    return pop

def day2(pop, r=5, p_i = .3, p_v =-1 ):
    #Group by status
    groups =[ x for x in pop.groupby('status') ] #pop.groupby('status')
    for idx, p in groups[0][1].iterrows():
    #for each infected person
        prob = p_v if 'vaccinated' in pop.columns and p_v != -1 and p['vaccinated'] == 1 else p_i  #not -1?
        #check if distance is in radius
        for _, z in groups[1][1].iterrows():
            if (p.x - z.x)** 2 +(p.y - z.y)**2 < r**2:
                #check for infection event
                if np.random.random() < prob:
                    #recor it and stop
                    pop.at[idx, 'status'] = 1
                    break
    # return dataframe
    return pop

    # pop[column].value():
        #
    #Group dataframe by status
    #for each non infected person
        #for each infected person
            #check if distance is in radius
                #check for an infection event
                    # record it and stop

Run some tests on your function to ensure it's working properly. A unit test is suggested below to get you started (change the name to match your function).

In [4]:
test1 = day2( pd.read_csv("lab2-sample.csv"), 5, 1)
#test1.sort_values(by = 'status', ascending = False)
assert( test1.status.value_counts()[1] == 13 )

---

Create a function to run a complete simulation. The function should take the following parameters:
1. A dataframe of the population to simulate
2. The number of days to simulate (default to 30)
3. The infection radius distance (default to 5)
4. The infection probability of non-vaccinated people (default to 3)
5. The vaccine effectiveness (default to 0)
6. The vaccine rate (default to 0)

The function should first make a copy of the population dataframe (use the DataFrame.copy() method). If the vaccine effectiveness is not 0, the population should first be vaccinated. The number of vaccinations should equal the number of non-infected individuals times the vaccine rate. The individuals to be vaccinated should be chosen at random. When vaccinating, create a `vaccinated` column and set vaccinated individuals to 1. After vaccination the function should call your single day simulation function for the number days specified in the parameter to this function. When calling the day function, pass in the appropriate arguments. To calculate the argument for the infection probability for vaccinated people, use the probability of infection for non-vaccinated people scaled by (1 - vaccine effectiveness). Return the modified population dataframe.

In [5]:
def sim(population, days = 30, r = 5, p_i = 0.3, v_eff = 0, v_rate = 0):
    pop = population.copy()
    if v_eff != 0:
        pop['vaccinated'] = 0
        h = pop[pop.status == 0].index
        v_idx = np.random.choice(h, size = int(len(h)*v_rate), replace = False)
        pop.loc[ v_idx, 'vaccinated' ] = 1
    pv = p_i * (1 - v_eff) if v_eff != 0 else -1
    for _ in range(days):
        pop = day2(pop, r, p_i, pv)
    return pop

Test your function by running it on "lab2-sample.csv" with and without vaccines. When testing vaccines try using a vaccine effectiveness and rate both set to 1, then confirm that no infections occur.

In [6]:
testvac1 = sim(pd.read_csv("lab2-sample.csv"), v_eff=1, v_rate=1)
assert( testvac1.status.value_counts()[1] == 2) # no one should be infected other than the starting 2

---

Create a function that runs iterations of an experiment on a data file and saves the number of infected indivduals for each run and returns these in a list. The first parameter should be the file (default to "lab2-sample.csv"), the second parameter should be the number of iterations (default 10) and the remaining parameters should match those required in your simulation function above (with the exception of the population dataframe).

In [7]:
def runs( f="lab2-sample.csv", n = 10, days = 30, r =5, p_i = .3, v_eff = 0 , v_rate = 0):
    return [int(sim(pd.read_csv(f), days, r, p_i, v_eff, v_rate).status.value_counts().get(1, 0)) for _ in range(n)]


---

Call your iterative function using defaults and save the results as a baseline.

In [8]:
baseline = runs()
baseline

[186, 180, 173, 189, 172, 173, 172, 173, 181, 172]

Call your iterative function using using a vaccine with 60% effectiveness and an 80% rate and save the results as vaccine 1.

In [9]:
vaccine1 = runs(v_eff = .6, v_rate = .8)
vaccine1

[108, 101, 103, 141, 143, 141, 131, 96, 111, 81]

Call your iterative function using using a vaccine with 80% effectiveness and a 60% rate and save the results as vaccine 2.

In [10]:
vaccine2 = runs(v_eff = .8, v_rate = .6)
vaccine2

[85, 138, 79, 84, 111, 97, 124, 100, 129, 67]

Run a two-sided proportions test comparing the proportion of the means of infections in the baseline and vaccine 1 to determine if they are significantly different.

In [11]:
count = [int(np.mean(baseline)), int(np.mean(vaccine1))]
nobs = [len(baseline) * 200, len(vaccine1) * 200] 
prop_var = (int(np.mean(baseline)) + int(np.mean(vaccine1))) / (sum(nobs))
proportion.proportions_ztest(count=count, nobs=nobs, value=0, alternative='two-sided', prop_var=prop_var)

(np.float64(3.768429515832426), np.float64(0.00016427785696122073))

Run the same test as above comparing vaccine 1 and vaccine 2.

In [12]:
count = [int(np.mean(vaccine1)), int(np.mean(vaccine2))]
nobs = [len(vaccine1) * 200, len(vaccine2) * 200] 
prop_var = (int(np.mean(vaccine1)) + int(np.mean(vaccine2))) / (sum(nobs))
proportion.proportions_ztest(count=count, nobs=nobs, value=0, alternative='two-sided', prop_var=prop_var)

(np.float64(0.9793898343236672), np.float64(0.3273873980640638))

Interpret the results of your two proportions test.

Looking at Basline vs Vaccine 1 output we can see that we have a high z stat meaing that im 4.39 standard deviations away from the mean difference of both data sets given we assume the null hypothesis is true. There is a huge difference between the groups. The p-value is extremely small, I would argue near zero which tells me to reject the null hypothesis and say there is a statistically significant diffference between baseline and vaccine1 groups.

For Vaccine 1 vs Vaccine 2 our z stat is much closer to zero meaning the groups are similar to each other (only 0.5 std dev apart), also supporting our claim the p value is large (61.4%) meaing there is a high probability that they are similar. We fail to reject the null hypothesis and can say that there is no statistically significant difference between the two vaccines.


Based on these results, find a vaccine effectiveness and rate that minimizes infections with the constraint that the sum of the effectiveness and rate must be 1.4 or less. (Both previous vaccine tests satisfy this constraint: .8 + .6 <= 1.4 and .6 + .8 <= 1.4). Show that your vaccine is significantly different than vaccine 1 and 2.

In [13]:
#There is a MUCH better way to do this but honestly I tried to make a function that stored means and iterated the effectiveness and rate but didn't work.
#In retrospect its very obvious that O10 would be the best its 100% effective...
O4 = runs(v_eff = 0.4, v_rate = 1.0)
print(np.mean(O4))

O5 = runs(v_eff = 0.5, v_rate = 0.9)
print(np.mean(O5))

O6 = runs(v_eff = 0.6, v_rate = 0.8)
print(np.mean(O6))

O7 = runs(v_eff = 0.7, v_rate = 0.7)
print(np.mean(O7))

O8 = runs(v_eff = 0.8, v_rate =  0.6)
print(np.mean(O8))

O9 = runs(v_eff = 0.9, v_rate = 0.5)
print(np.mean(O9))

O10 = runs(v_eff = 1.0, v_rate = 0.4)
print(np.mean(O10))

141.1
125.6
115.7
104.6
94.2
73.5
43.1


In [14]:
best_vaccine = runs(v_eff = 1.0, v_rate = .4)

count = [int(np.mean(best_vaccine)), int(np.mean(vaccine1))]
nobs = [len(best_vaccine) * 200, len(vaccine1) * 200] 
prop_var = (int(np.mean(vaccine1)) + int(np.mean(vaccine2))) / (sum(nobs))
print(proportion.proportions_ztest(count=count, nobs=nobs, value=0, alternative='two-sided', prop_var=prop_var))

count = [int(np.mean(best_vaccine)), int(np.mean(vaccine2))]
nobs = [len(best_vaccine) * 200, len(vaccine2) * 200] 
prop_var = (int(np.mean(best_vaccine)) + int(np.mean(vaccine2))) / (sum(nobs))
print(proportion.proportions_ztest(count=count, nobs=nobs, value=0, alternative='two-sided', prop_var=prop_var))

(np.float64(-5.0368620050931465), np.float64(4.732256841694787e-07))
(np.float64(-4.922755097515544), np.float64(8.533421788306673e-07))


Finally run your vaccine once on the entire population "lab2.csv" (this will take some time) and report whether you've saved the city (infection rate contained to 50%).

In [15]:
city = runs(f="lab2.csv", v_eff = 1.0, v_rate = 0.4 )
city

[438, 400, 481, 472, 442, 428, 499, 521, 419, 463]

In [16]:
if np.mean(city)/1000 <= 0.5:
    print("they survived")
else:
    print("zombies won")

they survived


---

### Submission Instructions

Be sure to ***SAVE YOUR WORK***!  

Next, select File -> Save and Export As ... -> HTML

Then submit your notebook HTML file to Canvas.